In [1]:
"""
- Primary Objective
 - When creating a premarket watchlist, I need the ability to determine which stock picks has the highest chance of sustained open market to midday move. 
- Define success definition
 - Largest 9:30 AM - 12:00 PM stock gainers and  losers
 - Highest option gainers and losers during 9:30 AM - 12:00 PM
 - If (CurrentOpen - PrevClose) is positive > than If (DayAvg. - CurrentOpen) is positive
- Analyze for patterns
 - Get spy, qqq, and dia to understand overall performance of the day
 - Get news 
"""

"""
- How to distinguish news movement vs non-news movement
"""

'\n- How to distinguish news movement vs non-news movement\n'

In [4]:
## Pull in raw price data
from datetime import datetime
from iexfinance.stocks import get_historical_data

token = "pk_6bae0caafd6942bcbfcb3374895fd4ae"

start = datetime(2021, 3, 1)
end = datetime(2021, 4, 1)
rawImport = get_historical_data("tsla", start, end, output_format = 'pandas', token = token)
filterImport = rawImport[['symbol','open','close','high','low','volume','label']]


In [2]:
post_data = []
for i in range(len(filterImport)):
    post_data.append([
        str(filterImport['symbol'][i]),
        str(filterImport['open'][i]),
        str(filterImport['close'][i]),
        str(filterImport['high'][i]),
        str(filterImport['low'][i]),
        str(filterImport['volume'][i]),
        str(filterImport['label'][i])
    ])
print(type(post_data))
print()


<class 'list'>


In [5]:
print(type(filterImport))
post_data = list(filterImport.itertuples(index=False))
print(type(post_data))
print(post_data)

<class 'pandas.core.frame.DataFrame'>
<class 'list'>
[Pandas(symbol='TSLA', open=690.11, close=718.43, high=872, low=685.05, volume=27136239, label='Mar 1, 21'), Pandas(symbol='TSLA', open=718.28, close=686.44, high=721.11, low=685, volume=23732158, label='Mar 2, 21'), Pandas(symbol='TSLA', open=687.99, close=653.2, high=700.7, low=651.705, volume=30207960, label='Mar 3, 21'), Pandas(symbol='TSLA', open=655.8, close=621.44, high=873.94, low=600, volume=65919530, label='Mar 4, 21'), Pandas(symbol='TSLA', open=626.06, close=597.95, high=627.8419, low=539.49, volume=89396459, label='Mar 5, 21'), Pandas(symbol='TSLA', open=600.55, close=563, high=620.125, low=558.79, volume=51786958, label='Mar 8, 21'), Pandas(symbol='TSLA', open=608.18, close=673.58, high=678.09, low=595.21, volume=67523328, label='Mar 9, 21'), Pandas(symbol='TSLA', open=700.3, close=668.06, high=717.85, low=655.06, volume=60605672, label='Mar 10, 21'), Pandas(symbol='TSLA', open=699.4, close=699.6, high=702.5, low=677.18

In [9]:
## Load raw price data to database
import psycopg2
port = 5432
user = 'postgres'
host = 'localhost'
database = 'Research'
password = 'One_Drive^@11'
try:
    connection = psycopg2.connect(
        user = user, 
        password = password, 
        host = host, 
        port = port, 
        database = database
    )

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

{'user': 'postgres', 'dbname': 'Research', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.2, compiled by Visual C++ build 1914, 64-bit',) 



In [10]:
db_schema = 'research_sch'
table = 'raw_historical_prices'
insertList = tuple(post_data)
columns = ['symbol','open','close','high','low','volume','date']
columnstring = " ("
endstring = "("
for item in columns:
    columnstring = columnstring + item + ","
    endstring = endstring + "%s,"
       
columnstring = columnstring[:-1]
columnstring = columnstring + ") "

endstring = endstring[:-1]
endstring = endstring + ");"
query_string = 'INSERT INTO ' + db_schema + '.' + table + columnstring +' VALUES ' + endstring

print(query_string)

# Execute query
try:
    cursor.executemany(query_string, insertList)
    connection.commit()
except RuntimeError as e:
    print(e)
        
cursor.close()


INSERT INTO research_sch.raw_historical_prices (symbol,open,close,high,low,volume,date)  VALUES (%s,%s,%s,%s,%s,%s,%s);


In [17]:
from datetime import timedelta
from datetime import datetime
from datetime import date
start = date.today() - timedelta(days=30)
end = date.today()
print(start)
print(end)

2021-03-04
2021-04-03


In [1]:
import csv
import pandas as pd
file = 'C:\\Users\\yimin\\Documents\\Github Repos\\Financial-Services\\Research\\files\\Tracker.csv'
df = pd.read_csv(file)
post_data = list(df.itertuples(index=False))
print(type(post_data))

<class 'list'>


In [2]:
print(post_data)

L', Name='Apple', Price=123.01, Bid=123.0, Ask=123.01, _5=0.01, _6=0.01, _7=145.09, _8=59.225, _9=124.18, _10=122.49, Volume=73851286.0, _12=2064935808000.0, Sector='Information Technology'), Pandas(Symbol='ABBV', Name='AbbVie', Price=108.45, Bid=108.01, Ask=109.73, _5=0.3, _6=0.28, _7=113.41, _8=71.43, _9=108.9822, _10=107.32, Volume=4112996.0, _12=191395358552.0, Sector='Health Care'), Pandas(Symbol='ACN', Name='Accenture', Price=278.49, Bid=278.22, Ask=278.49, _5=2.14, _6=0.77, _7=281.3, _8=148.28, _9=278.77, _10=276.065, Volume=1466011.0, _12=177021390120.0, Sector='Information Technology'), Pandas(Symbol='ACTC', Name='ArcLight Clean Transition', Price=18.38, Bid=18.3, Ask=18.74, _5=0.48, _6=2.68, _7=31.06, _8=9.74, _9=19.75, _10=18.2209, Volume=896469.0, _12=637556250.0, Sector=nan), Pandas(Symbol='ADI', Name='Analog Devices', Price=160.38, Bid=158.21, Ask=161.0, _5=5.3475, _6=3.45, _7=164.4, _8=82.9, _9=160.49, _10=156.89, Volume=3332383.0, _12=59163178341.0, Sector='Information 